In [1]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers import Bidirectional
from keras.layers.convolutional import MaxPooling1D


Using TensorFlow backend.


# DATA PREPARATION: Split a univariate sequence into samples

In [2]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
# find the end of this pattern
        end_ix = i + n_steps
# check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
# gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# Define input sequence

In [3]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)

In [4]:
X,y

(array([[10, 20, 30],
        [20, 30, 40],
        [30, 40, 50],
        [40, 50, 60],
        [50, 60, 70],
        [60, 70, 80]]), array([40, 50, 60, 70, 80, 90]))

In [5]:
X.shape

(6, 3)

In [6]:
# Reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X)

[[[10]
  [20]
  [30]]

 [[20]
  [30]
  [40]]

 [[30]
  [40]
  [50]]

 [[40]
  [50]
  [60]]

 [[50]
  [60]
  [70]]

 [[60]
  [70]
  [80]]]


# Vanilla LSTM

In [7]:
# Define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [8]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [9]:
# demonstrate prediction
x_input = array([70,80,90])
x_input = x_input.reshape((1, n_steps, n_features))

In [10]:
yhat = model.predict(x_input, verbose=0)
print(yhat)


[[ 102.17403412]]


# Stacked LSTM

In [11]:
# Define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [12]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [13]:
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))

In [14]:
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[ 102.71408081]]


# Bi-Directional LSTM

In [15]:
# define model
model = Sequential()
model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [16]:
# fit model
model.fit(X, y, epochs=200, verbose=0)

In [17]:
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))

In [18]:
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[ 101.26158142]]


# CNN LSTM

In [19]:
# choose a number of time steps
n_steps = 4
# split into samples
X, y = split_sequence(raw_seq, n_steps)


In [20]:
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, n_steps, n_features))

In [21]:
# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [22]:
# fit model
model.fit(X, y, epochs=500, verbose=0)

In [23]:
# demonstrate prediction
x_input = array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, n_steps, n_features))

In [24]:
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[ 102.89810181]]


# Conv LSTM

In [25]:
# choose a number of time steps
n_steps = 4
# split into samples
X, y = split_sequence(raw_seq, n_steps)

In [26]:
# reshape from [samples, timesteps] into [samples, timesteps, rows, columns, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))

In [27]:
# Define model
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model.add(Flatten())
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [28]:
# fit model
model.fit(X, y, epochs=500, verbose=0)

In [29]:
# demonstrate prediction
x_input = array([60, 70, 80, 90])
x_input = x_input.reshape((1, n_seq, 1, n_steps, n_features))

In [30]:
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[ 102.53213501]]
